In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("corona.csv")
df.head()

C:\Users\Arijit Dalui\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,30-04-2020,0,0,0,0,0,negative,None,female,Other
1,30-04-2020,1,0,0,0,0,negative,None,female,Other
2,30-04-2020,0,1,0,0,0,negative,None,male,Other
3,30-04-2020,1,0,0,0,0,negative,None,female,Other
4,30-04-2020,1,0,0,0,0,negative,None,male,Other


In [3]:
df.replace('negative', 0,inplace=True)
df.replace('positive', 1,inplace=True)
df.replace('other', 0,inplace=True)

In [4]:
df.age_60_and_above.unique()

array(['None', 'Yes', 'No'], dtype=object)

In [5]:
df.gender.unique()

array(['female', 'male', 'None'], dtype=object)

In [6]:
df.corona_result.unique()

array([0, 1], dtype=int64)

In [7]:
df.drop('test_date',axis=1,inplace=True)

In [8]:
df.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,0,0,0,0,0,0,None,female,Other
1,1,0,0,0,0,0,None,female,Other
2,0,1,0,0,0,0,None,male,Other
3,1,0,0,0,0,0,None,female,Other
4,1,0,0,0,0,0,None,male,Other


In [9]:
df['cough'].unique()

array([0, 1, '0', '1', 'None'], dtype=object)

In [10]:
def normalise(x):
    if type(x)==str:
        return int(x) if x != "None" else 0
    return x
for col in ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache']:
    df.loc[:, col] = df[col].apply(normalise)
df

,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,0,0,0,0,0,0,None,female,Other
1,1,0,0,0,0,0,None,female,Other
2,0,1,0,0,0,0,None,male,Other
3,1,0,0,0,0,0,None,female,Other
4,1,0,0,0,0,0,None,male,Other
...,...,...,...,...,...,...,...,...,...
278843,0,0,0,0,0,0,None,None,Other
278844,0,0,0,0,0,0,None,None,Other
278845,0,0,0,0,0,1,None,None,Contact with confirmed
278846,0,0,0,0,0,0,None,None,Other


In [11]:
x=df.iloc[:, 0:5]
y=df.iloc[:, 6:9]
new=pd.concat([x,y],axis=1)
x=new
y=df.iloc[:, 5:6].values
x

,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,test_indication
0,0,0,0,0,0,None,female,Other
1,1,0,0,0,0,None,female,Other
2,0,1,0,0,0,None,male,Other
3,1,0,0,0,0,None,female,Other
4,1,0,0,0,0,None,male,Other
...,...,...,...,...,...,...,...,...
278843,0,0,0,0,0,None,None,Other
278844,0,0,0,0,0,None,None,Other
278845,0,0,0,0,0,None,None,Contact with confirmed
278846,0,0,0,0,0,None,None,Other


In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [5,6,7])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [15]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(x_train, y_train)

C:\Users\Arijit Dalui\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(random_state=0)

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[65506   513]
 [ 1556  2137]]


0.9703207482212531

In [18]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(x_train, y_train)

C:\Users\Arijit Dalui\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[65345   674]
 [ 1683  2010]]


0.9661894652283681

In [20]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(x_train, y_train)

C:\Users\ARIJIT~1\AppData\Local\Temp/ipykernel_14588/3745342088.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(x_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[65511   508]
 [ 1558  2135]]


0.9703637824190957

In [22]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(x_train, y_train)

C:\Users\Arijit Dalui\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[63819  2200]
 [ 1293  2400]]


0.949893848978655

In [24]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(x_train, y_train)

C:\Users\ARIJIT~1\AppData\Local\Temp/ipykernel_14588/3745342088.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(x_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[65511   508]
 [ 1558  2135]]


0.9703637824190957

In [30]:
x_test[0]

array([ 1.10406051, -0.91694682, -0.31941606, -0.27483978,  1.06644841,
       -0.92647663, -0.3171658 , -0.19923992,  0.38583889, -0.42292857,
       -0.29085268, -0.08279883, -0.07552016, -0.09325318])